In [28]:
import ujson

with open('user_project_summaries.json') as o:
    summaries = ujson.load(o)

## Project names

### Morphological analysis

Use a morphological analyzer to learn to split project names.

In [29]:
with open('projectnames.txt', 'w') as o:
    for userid in summaries:
        for projectid in summaries[userid]:
            o.write(summaries[userid][projectid]['**Project Name']+' ')

In [3]:
import morfessor
io = morfessor.MorfessorIO()
train_data = io.read_corpus_file('projectnames.txt')
model_tokens = morfessor.BaselineModel()
model_tokens.load_data(train_data)
model_tokens.train_batch()

...........................................................
...........................................................
...........................................................
...........................................................
...........................................................
...........................................................


(6, 900608.304622147)

In [30]:
pnames = open('projectnames.txt').read().split()

In [5]:
segpnames = [model_tokens.viterbi_segment(word) for word in pnames]

In [6]:
segpnames[:50]

[(['Phone', 'Access', 'Lab'], 27.10542653842551),
 (['BtCarEmuLab'], 11.531074228319381),
 (['BN103096', 'Voice', 'Dict'], 27.92975191766923),
 (['BN103096', 'Mole'], 17.13937566778919),
 (['BN103096', 'L', 'AP', '01'], 34.40717377259127),
 (['BN103096', 'Lab0305'], 19.801963494814643),
 (['BtOX', 'Game', 'Lab', 'V1'], 33.641519730322656),
 (['Screen', '4'], 14.458709533413161),
 (['BN', '1', '03', '09', '7', 'EX05', '07', '02'], 58.17916833630658),
 (['BN103096', 'T2', 'S'], 26.255775484097555),
 (['BN103096', '1'], 14.11725058287879),
 (['BN103299', 'Lab0305'], 21.961447744168012),
 (['Block', '_', 'Lab', '00'], 30.00788550473449),
 (['BN103096'], 9.371589978966009),
 (['BN103096', 'OddEven'], 19.108816314254696),
 (['BN103096', 'ST', 'ER'], 26.67698894917386),
 (['BN103096', 'AA', '01'], 24.80347882992711),
 (['BN103096', 'Sensor', '1'], 21.88503627170197),
 (['BN10302M', 'D', '01'], 25.058725626625737),
 (['BN103096', 'EX05', '07', '02'], 32.263342038286794),
 (['BN10302M', 'D', '0

Does this give anything reasonable? Remains to be explored.

### Clustering

Cluster the project names using edit distance.

In [31]:
import leven
from sklearn.cluster import AffinityPropagation
import numpy as np

pnameset = list(set(pnames))[:5000]   # will increase, but clustering takes longer

lev_aff = np.zeros((len(pnameset), len(pnameset)))
for i, pnamei in enumerate(pnameset):
    if i%10==0:
        print '.',
    for j, pnamej in enumerate(pnameset):
        if j>i:
            lev_aff[i, j] = -1*leven.levenshtein(pnamei, pnamej)
        elif j==i:
            lev_aff[i, j] = 0
        else:
            lev_aff[i, j] = lev_aff[j, i]
print 'Computed pairwise edit distances'

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [32]:
model = AffinityPropagation(affinity="precomputed")
model.fit(lev_aff)
print 'Fit affinity propagator'

Fit affinity propagator


In [13]:
exemplars = defaultdict(list)
for i, clustid in enumerate(model.labels_):
    exem = model.cluster_centers_indices_[clustid]
    exemplars[pnameset[exem]].append(pnameset[i])
print exemplars

defaultdict(<type 'list'>, {'android_bot_clock_ten': ['android_bot_clock_ten'], 'AnimationArrowsandBallsSprite': ['AnimationArrowsandBallsSprite'], 'Moore_WorkshopSelector': ['Moore_WorkshopSelector', 'JK_WorkshopSelector'], 'examen1': ['exex1', 'RGBsend', 'hellomeow', 'examen3', 'examen2', 'examen1', 'examenn', 'Text_compare_ex', 'exexexex', 'ExamLb', 'flaggen2f1', 'exam_1'], 'CalculadoraBasica': ['CalculadoraBasica', 'calculadoraMariaJ', 'Calculadora66final', 'CalculadoradeRaces'], 'Factorialdeunnumero_InigoN': ['Factorialdeunnumero_InigoN'], 'hallokatze': ['hello_david', 'hallokatze', 'hello_purr', 'a8_Ball_Game', 'Sample_Book_Store', 'hallokatze2', 'hallokatze4', 'WhatDoesItSay', 'paletka_gdansk'], 'lesson8a': ['lesson8a', 'lesson8b', 'GuessWhat', 'DesignGedean', 'sesion7', 'Test_only', 'AI2_Lesson1_ex1', 'frmEnonce3a', 'anaysofia'], 'ESTUDIO5PRUEBA2': ['EJERCICIO3GUIA2', 'ESTUDIO5PRUEBA2'], 'midexam_1': ['midexam_5', 'midexam_4', 'midexam_3', 'midexam_2', 'midexam_1'], 'whereamIco

## Language Identification from Project and Variable Names

Predict the native language of a user from their project/variable names and strings.

In [6]:
import langid

user_tokens = defaultdict(list)
for userid in summaries:
    for projectid in summaries[userid]:
        numscreens = summaries[userid][projectid]['*Number of Screens']
        for screenid in range(numscreens):
            screentitle = 'Screen'+str(screenid+1)
            if screentitle in summaries[userid][projectid]:
                # blocks
                if summaries[userid][projectid][screentitle]['Blocks']=='NO BLOCKS':
                    continue
                blocks = summaries[userid][projectid][screentitle]['Blocks']
                if blocks['Active Blocks']=='NO ACTIVE BLOCKS':
                    continue
                for namesp in ['Global Variable Names', 'Procedure Names', 'Procedure Parameter Names', 'Strings']:
                    for vname in blocks['Active Blocks'][namesp]:
                        user_tokens[userid].append(vname)
                # components
                if summaries[userid][projectid][screentitle]['Components']=='N':
                    continue
                for sname in summaries[userid][projectid][screentitle]['Components']['Strings']:
                    user_tokens[userid].append(sname)
            

In [ ]:
from collections import defaultdict

user_langs = defaultdict(lambda : defaultdict(float))
for userid in user_tokens:
    if len(user_tokens[userid])<50:
        continue
    user_langs[userid] = langid.classify(' '.join(user_tokens[userid]))[0]
            

In [26]:
langcounts = defaultdict(int)
for userid in user_langs:
    langcounts[user_langs[userid]]+=1
    if user_langs[userid]=='it':
        print userid, ' '.join(user_tokens[userid])

00061 global tempo tempo Attendi prego Caricamento in corso.... Start global punti punti CancellaText GeneraNumeri null - Numeri indovinati:    other currentX edge DecBin Menu convertitori Decimale --> Binario ListaEstratta global ListaDati QueryFinale TestFinale global risul1T global stringaPicker stringaPicker global stringaHelp time stringaHelp responseContent ListaDati global risul strFindMatch risul risul1T responseCode global time global ListaEstratta global TestFinale global strFindMatch ImpostaPicker impostaText CreaQuery   null OK ) ( * , \nTempo:   http://www.syncandroid.altervista.org/showmatch.php Errore end Risultato:   \nRisultato 1° tempo:   Text for Label1 global risultato risultato + null - ? Copiato !! global time_ore global time_min time_ore global time_sec time_sec time_min AzzeraTutto trueOrFalse functionName global boolServer boolServer errorNumber textToPrint message disconnect send init trueOrFalse connected print message textToPrint message trueOrFalse textToPr

In [12]:
langcounts

defaultdict(int,
            {'ar': 4,
             'br': 1,
             'ca': 10,
             'de': 12,
             'el': 6,
             'en': 422,
             'es': 105,
             'et': 1,
             'fi': 1,
             'fr': 6,
             'gl': 1,
             'he': 4,
             'it': 22,
             'ja': 4,
             'ko': 81,
             'la': 12,
             'lb': 1,
             'lv': 1,
             'ms': 1,
             'nl': 5,
             'no': 3,
             'pl': 4,
             'pt': 14,
             'ru': 1,
             'se': 1,
             'sq': 1,
             'te': 1,
             'th': 3,
             'tr': 4,
             'vi': 2,
             'zh': 238})